# Coherent info

In [2]:
import numpy as np
from collections import Counter
from typing import Tuple, Iterable, Dict, List
from coherentinfo.moebius import MoebiusCode, MoebiusCodeOddPrime, MoebiusCodeQubit
from coherentinfo.linalg import (finite_field_gauss_jordan_elimination,
                                 finite_field_matrix_rank,
                                 finite_field_inverse,
                                 is_prime)
from coherentinfo.postprocess import aggregate_data, compute_conditional_entropy_term, conditional_entropy

from numpy.typing import NDArray

from coherentinfo.errormodel import ErrorModel, ErrorModelBernoulli, ErrorModelBernoulliJax
# import galois
import scipy
import timeit
import jax
import jax.numpy as jnp

In [3]:
length = 5
width = 5
p = 3
moebius_code = MoebiusCodeOddPrime(length=length, width=width, d=2 * p)
moebius_code_qubit = MoebiusCodeQubit(length=length, width=width)
h_z = moebius_code.h_z
h_x = moebius_code.h_x
logical_x = moebius_code.logical_x
logical_z = moebius_code.logical_z

In [3]:
# gamma = 0.5
p_error = 0.21
# poisson_em_moebius = ErrorModelPoisson(moebius_code.num_edges, 2 * p, gamma)
em_moebius_qubit = ErrorModelBernoulli(moebius_code.num_edges, 2, p_error)
em_moebius_qubit_jax = ErrorModelBernoulliJax(
    moebius_code.num_edges, 2, p_error
)

In [ ]:
num_samples = 1_000_000
vertex_result = moebius_code_qubit.compute_batched_vertex_syndrome_chi_z(
    num_samples, em_moebius_qubit_jax)

: 

In [1]:
_, vertex_probs = aggregate_data(vertex_result)
del vertex_result

NameError: name 'aggregate_data' is not defined

In [6]:
vertex_conditional_entropy = conditional_entropy(vertex_probs)
del vertex_probs

In [ ]:
plaquette_result = moebius_code_qubit.compute_batched_plaquette_syndrome_chi_x(
    num_samples, em_moebius_qubit_jax)

In [8]:
_, plaquette_probs = aggregate_data(plaquette_result)
del plaquette_result

In [9]:
plaquette_conditional_entropy = conditional_entropy(plaquette_probs)
del plaquette_probs

In [10]:
coherent_info = 1.0 - vertex_conditional_entropy - plaquette_conditional_entropy
print("Coherent information = {}".format(coherent_info))

Coherent information = 0.549363374710083
